# Import Statements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, max_error, median_absolute_error
from sklearn.feature_selection import mutual_info_regression
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import csv
import os
import psycopg2
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


# Transfer the Data to Postgresql

#### Let's Start by storing our data from the csv file in a dataframe
#### After inspecting the data we notice that the columns are seperated with a ';' semi colon, to solve this we use delimiter to seperate the columns in our dataframe. 
#### The print statements will give us a glimpse of the code structure and a brief summary about each columns statistics.

In [ ]:
df = pd.read_csv("./docs/data.csv", delimiter=';')
print(df.head(10))  # Display first few rows
print(df.describe())  # Display summary statistics

### Below we will have two functions that will do the following:
1. The first function (load_data_to_postgres_docker) will load the data from the csv file into the database
2. The second function (add_new_student) will add a new student row with information provided to the postgres database.

In [ ]:
def load_data_to_postgres_docker(table_name, user, password, host='mypostgres', port=5432, database_name='student_grades'):
    # Create engine with SQLAlchemy
    password = quote_plus(password)
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database_name}')

    # Transfer data to database
    df.to_sql(table_name, engine, index=False, if_exists='replace')

# load_data_to_postgres_docker('student_grades_table', 'leonphoenix21', 'vtlt123@dck')


In [ ]:
def add_new_student(student_info, table_name, database_name, user, password, host='127.0.0.1', port=5432):
    # Create engine with SQLAlchemy
    password = quote_plus(password)
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database_name}')

    # Convert student_info to DataFrame
    new_student_data = pd.DataFrame([student_info])

    # Append new student data to the PostgreSQL table
    new_student_data.to_sql(table_name, engine, index=False, if_exists='append')

new_student_info = {
    'school': 'GP',
    'sex': 'M',
    'age': 16,
    'address': 'U',
    'famsize': 'LE3',
    'Pstatus': 'T',
    'Medu': 4,
    'Fedu': 3,
    'Mjob': 'teacher',
    'Fjob': 'services',
    'reason': 'course',
    'guardian': 'mother',
    'traveltime': 3,
    'studytime': 2,
    'failures': 0,
    'schoolsup': 'no',
    'famsup': 'yes',
    'paid': 'no',
    'activities': 'yes',
    'nursery': 'yes',
    'higher': 'yes',
    'internet': 'yes',
    'romantic': 'no',
    'famrel': 5,
    'freetime': 4,
    'goout': 3,
    'Dalc': 1,
    'Walc': 2,
    'health': 1,
    'absences': 2,
    'G1': 15,
    'G2': 14,
    'G3': 15
}

# add_new_student(new_student_info, 'student_grades_table', 'student_grades', 'leonphoenix21', 'vtlt123@dck')

#### Now we are going to Clean, process and explore the data, by calculating the frequencies and mutual information score for each feature as well creating visualizations that will give an insight into the correlation between features. Allowing us to analyze and better understand the relationship of the features to the target feature.
#### The Code Blocks below should accomplish the following tasks:
+ Handle missing values 
* We will check for any inconsistent data entry any duplicates e.t.c
<br>

### Handle missing values.
#### First Section here checks for the missing values and then we print all the columns and the number of missing values since we have a smaller database. 

In [ ]:
# Checking for missing values
missing_values = df.isnull().sum()
print(missing_values)

# how many total missing values do we have?
total_cells = np.product(df.shape)
total_missing = missing_values.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

#### Check for duplicated rows
Now we move on to checking for duplicate rows in our dataframe and drop them if any

In [ ]:
print(df.duplicated().value_counts())
df = df.drop_duplicates()
print(df)

# Feature Selection 

Now we are about to Train our model. First in the code block below I create a copy of the data without the G3 column and we then clone the G3 column in the y-variable so that we can compare our predictions to the actual results from our target variable

In [ ]:
X = df.drop("G3", axis = 1)
print(X.describe())
y = df["G3"]

print(X.shape)
print(y.shape)

So Here I am defining my categorical columns and I am going to encode those columns using a label encoder. I am making a note to encode the X variables and not the main (df) dataframe variables.

In [ ]:
# Define the categorical columns
categorical_columns = [
    'school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian',
    'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic'
]

# Apply label encoding to categorical columns in X
label_encoder = LabelEncoder()
for column in categorical_columns:
    X[column] = label_encoder.fit_transform(X[column])

# Display the DataFrame after label encoding
print(X.head(1))

Below I plan to create visualizations that will help us with our feature selection by seeing the correlation between the features, the spread of each feature and the their shape.

In [ ]:
EDA_df = X.copy()

corr_matrix = EDA_df.corr()

plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix,  cmap='coolwarm', annot_kws={"size": 5})
plt.title("Correlation Heatmap")
plt.xlabel("Variables")
plt.ylabel("Variables")
plt.show()

In [ ]:
X.hist(bins=20, figsize=(15, 10))

After looking through the visualizations and exploring the data, I took a note of the performance of all the features and decided to create 10 new features as well as 10 feature subsets to evaluate and find the best performing model.

In [ ]:
# Create new features
X['academic_performance'] = X['G1'] + X['G2']
X['study_engagement'] = X['studytime'] * X['absences']
X['social_life'] = X['goout'] * X['freetime']
X['family_support'] = X['famsup'] * X['famrel']
X['health_and_activities'] = X['health'] * X['activities']
X['alcohol_consumption'] = X['Dalc'] + X['Walc']
X['study_habits'] = X['studytime'] * X['failures']
X['parental_education'] = X['Medu'] + X['Fedu']
X['internet_higher_edu'] = X['internet'] * X['higher']
X['Walc_health'] = X['Walc'] * X['health']



# All discrete features should now have integer dtypes
discrete_features = X.dtypes == int


Now that we have created new features we can calculate our mutual information score and create a visualization for each feature to find the best performing features to use that information for creating our featuresets

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
print(mi_scores)

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores, color='salmon')
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

# Assuming mi_scores is the mutual information scores you calculated
plt.figure(figsize=(10, 8))
plot_mi_scores(mi_scores)
plt.show()

After taking a look at the MI scores it's apparent that only a few features are highly dependent on our target variable (G3). So using that information we can now create an array of feature sets to test against our model. I chose to use the high performing features first and then to add more features to test which one work best.

In [ ]:
# Create a list of subsets to evaluate
featureSets = [
  ['G2', 'academic_performance', 'G1', 'study_engagement'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'romantic'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob', 'Walc'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob', 'Walc', 'Fjob'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob', 'Walc', 'Fjob', 'sex'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob', 'Walc', 'Fjob', 'sex', 'alcohol_consumption'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob', 'Walc', 'Fjob', 'sex', 'alcohol_consumption', 'paid'],
  ['G2', 'academic_performance', 'G1', 'study_engagement', 'failures', 'Mjob', 'Walc', 'Fjob', 'sex', 'alcohol_consumption', 'paid', 'schoolsup']
]

# Model Training 

Now that we have created new features and tested the performance and selected our features for our featureset. We will now be training our data using 3 different models to see which model will perform best with our features.


In [ ]:

results_df = pd.DataFrame(columns=['Feature Set', 'Model', 'r2', 'MSE', 'mean_absolute_percentage', 'max_error', 'median_absolute_error'])

for features in featureSets:
    try:
        X_features = X[features]
        print(X_features.head(1))

        # Scale features
        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X_features), columns=X_features.columns)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        # Define models
        models = {
            'KNN Regressor': KNeighborsRegressor(),
            'Linear SVR': SVR(),
            'Ridge Regressor': Ridge()
        }

        # Train and evaluate models
        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mean_absolute_percentage = mean_absolute_error(y_test, y_pred)
            max_err = max_error(y_test, y_pred)
            medianae = median_absolute_error(y_test, y_pred)

            results_df.loc[len(results_df.index)] = [', '.join(features), model_name, r2, mse, mean_absolute_percentage, max_err, medianae]

    except Exception as e:
        print(f"Error processing feature set {', '.join(features)}: {e}")

# Display results
print(results_df)

Now that we have trained our trained our models on our feature sets lets create some visualizations displaying the model performance performance by measuring different important metrics that tell us how accurate our predictions are. We can gain insights from this process to understand how we can fine-tune the model for enhanced performance.

In [ ]:
# Create a figure with subplots
plt.figure(figsize=(25,15))
plt.suptitle("Mean stats per model type")

# Subplot 1: Mean Absolute Error
plt.subplot(1, 5, 1)
plt.bar(results_df['Feature Set'], results_df['mean_absolute_percentage'])
plt.ylim(0.1)
plt.title("Mean Absolute Error")
plt.xticks(rotation='vertical')

# Subplot 2: r2
plt.subplot(1, 5, 2)
plt.bar(results_df['Feature Set'], results_df['r2'])
plt.ylim(0.6)
plt.title("r2")
plt.xticks(rotation='vertical')

# Subplot 3: Mean Squared Error
plt.subplot(1, 5, 3)
plt.bar(results_df['Feature Set'], results_df['MSE'])
plt.title("Mean Squared Error")
plt.xticks(rotation='vertical')

# Subplot 4: Max Error
plt.subplot(1, 5, 4)
plt.bar(results_df['Feature Set'], results_df['max_error'])
plt.title("Max Error")
plt.xticks(rotation='vertical')

# Subplot 5: Median Absolute Error
plt.subplot(1, 5, 5)
plt.bar(results_df['Feature Set'], results_df['median_absolute_error'])
plt.title("Median Absolute Error")
plt.xticks(rotation='vertical')

# Show the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  
plt.show()


In [ ]:
# Create subplots
fig, axes = plt.subplots(5, 1, figsize=(15, 24), sharex=True)

# Bar plot for R-squared (r2)
sns.barplot(x='Model', y='r2', hue='Feature Set', data=results_df, ax=axes[0])
axes[0].set_ylabel('R-squared (r2) Value')
axes[0].set_title('Comparison of R-squared Values by Feature Set and Model')

# Bar plot for Mean Squared Error (MSE)
sns.barplot(x='Model', y='MSE', hue='Feature Set', data=results_df, ax=axes[1])
axes[1].set_ylabel('Mean Squared Error (MSE) Value')
axes[1].set_title('Comparison of MSE Values by Feature Set and Model')

# Bar plot for Mean Absolute Percentage Error (MAPE)
sns.barplot(x='Model', y='mean_absolute_percentage', hue='Feature Set', data=results_df, ax=axes[2])
axes[2].set_ylabel('Mean Absolute Percentage Error (MAPE) Value')
axes[2].set_title('Comparison of MAPE Values by Feature Set and Model')

# Bar plot for Max Error
sns.barplot(x='Model', y='max_error', hue='Feature Set', data=results_df, ax=axes[3])
axes[3].set_ylabel('Max Error Value')
axes[3].set_title('Comparison of Max Error Values by Feature Set and Model')

# Bar plot for Median Absolute Error
sns.barplot(x='Model', y='median_absolute_error', hue='Feature Set', data=results_df, ax=axes[4])
axes[4].set_xlabel('Model')
axes[4].set_ylabel('Median Absolute Error Value')
axes[4].set_title('Comparison of Median Absolute Error Values by Feature Set and Model')

# Move the legend to the upper right corner
for ax in axes:
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

# Show the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  
plt.show()


# Model Tuning and Evaluation

After seeing the performance of the model, We can now fine tune the hyper-paramaters. I am first going to fine tune the parameters with Grid Search, and I chose this because our dataset isn't huge and the grid search should be a good start. 

In [ ]:


# Define the parameter grid for KNN Regressor
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_results_df = pd.DataFrame(columns=['Feature Set', 'Model', 'r2', 'MSE', 'mean_absolute_percentage', 'max_error', 'median_absolute_error'])

# Iterate through feature sets
for features in featureSets:
    try:
        X_features = X[features]

        # Scale features
        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X_features), columns=X_features.columns)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        # Create KNN Regressor model
        knn_model = KNeighborsRegressor()

        # Create GridSearchCV object
        grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='neg_mean_squared_error')

        # Perform GridSearchCV
        grid_search.fit(X_train, y_train)

        # Get the best hyperparameters
        best_params = grid_search.best_params_

        # Use the best hyperparameters to train the final model
        final_model = KNeighborsRegressor(**best_params)
        final_model.fit(X_train, y_train)

        # Evaluate the final model
        y_pred = final_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mean_absolute_percentage = mean_absolute_error(y_test, y_pred)
        max_err = max_error(y_test, y_pred)
        medianae = median_absolute_error(y_test, y_pred)

        # Log results
        grid_results_df.loc[len(grid_results_df.index)] = [', '.join(features), 'KNN Regressor', r2, mse, mean_absolute_percentage, max_err, medianae]

    except Exception as e:
        print(f"Error processing feature set {', '.join(features)}: {e}")

# Display results
print(grid_results_df)


In [ ]:
# Create subplots
fig, axes = plt.subplots(5, 1, figsize=(15, 24), sharex=True)

# Bar plot for R-squared (r2)
sns.barplot(x='Model', y='r2', hue='Feature Set', data=grid_results_df, ax=axes[0])
axes[0].set_ylabel('R-squared (r2) Value')
axes[0].set_title('Comparison of R-squared Values by Feature Set and Model (GridSearchCV)')

# Bar plot for Mean Squared Error (MSE)
sns.barplot(x='Model', y='MSE', hue='Feature Set', data=grid_results_df, ax=axes[1])
axes[1].set_ylabel('Mean Squared Error (MSE) Value')
axes[1].set_title('Comparison of MSE Values by Feature Set and Model (GridSearchCV)')

# Bar plot for Mean Absolute Percentage Error (MAPE)
sns.barplot(x='Model', y='mean_absolute_percentage', hue='Feature Set', data=grid_results_df, ax=axes[2])
axes[2].set_ylabel('Mean Absolute Percentage Error (MAPE) Value')
axes[2].set_title('Comparison of MAPE Values by Feature Set and Model (GridSearchCV)')

# Bar plot for Max Error
sns.barplot(x='Model', y='max_error', hue='Feature Set', data=grid_results_df, ax=axes[3])
axes[3].set_ylabel('Max Error Value')
axes[3].set_title('Comparison of Max Error Values by Feature Set and Model (GridSearchCV)')

# Bar plot for Median Absolute Error
sns.barplot(x='Model', y='median_absolute_error', hue='Feature Set', data=grid_results_df, ax=axes[4])
axes[4].set_xlabel('Model')
axes[4].set_ylabel('Median Absolute Error Value')
axes[4].set_title('Comparison of Median Absolute Error Values by Feature Set and Model (GridSearchCV)')

# Move the legend to the upper right corner
for ax in axes:
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

# Show the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the layout for better visibility
plt.show()


I decided to write another function to do a randomized hyper-parameter tuning. I just want to compare the result to that of the GridSearch and see which is the better performing tuning model.

In [ ]:


# Define the parameter distribution for KNN Regressor
param_dist = {
    'n_neighbors': randint(1, 10),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

random_results_df = pd.DataFrame(columns=['Feature Set', 'Model', 'r2', 'MSE', 'mean_absolute_percentage', 'max_error', 'median_absolute_error'])

# Iterate through feature sets
for features in featureSets:
    try:
        X_features = X[features]

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X_features), columns=X_features.columns)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        knn_model = KNeighborsRegressor()

        # Create RandomizedSearchCV object
        random_search = RandomizedSearchCV(knn_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error')

        # Perform RandomizedSearchCV
        random_search.fit(X_train, y_train)

        # Get the best hyperparameters
        best_params = random_search.best_params_

        # Use the best hyperparameters to train the final model
        final_model = KNeighborsRegressor(**best_params)
        final_model.fit(X_train, y_train)

        # Evaluate the final model
        y_pred = final_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mean_absolute_percentage = mean_absolute_error(y_test, y_pred)
        max_err = max_error(y_test, y_pred)
        medianae = median_absolute_error(y_test, y_pred)

        # Log results
        random_results_df.loc[len(random_results_df.index)] = [', '.join(features), 'KNN Regressor', r2, mse, mean_absolute_percentage, max_err, medianae]

    except Exception as e:
        print(f"Error processing feature set {', '.join(features)}: {e}")

# Display results
print(random_results_df)


In [ ]:
# Create subplots
fig, axes = plt.subplots(5, 1, figsize=(15, 24), sharex=True)

# Bar plot for R-squared (r2)
sns.barplot(x='Model', y='r2', hue='Feature Set', data=random_results_df, ax=axes[0])
axes[0].set_ylabel('R-squared (r2) Value')
axes[0].set_title('Comparison of R-squared Values by Feature Set and Model (RandomizedSearchCV)')

# Bar plot for Mean Squared Error (MSE)
sns.barplot(x='Model', y='MSE', hue='Feature Set', data=random_results_df, ax=axes[1])
axes[1].set_ylabel('Mean Squared Error (MSE) Value')
axes[1].set_title('Comparison of MSE Values by Feature Set and Model (RandomizedSearchCV)')

# Bar plot for Mean Absolute Percentage Error (MAPE)
sns.barplot(x='Model', y='mean_absolute_percentage', hue='Feature Set', data=random_results_df, ax=axes[2])
axes[2].set_ylabel('Mean Absolute Percentage Error (MAPE) Value')
axes[2].set_title('Comparison of MAPE Values by Feature Set and Model (RandomizedSearchCV)')

# Bar plot for Max Error
sns.barplot(x='Model', y='max_error', hue='Feature Set', data=random_results_df, ax=axes[3])
axes[3].set_ylabel('Max Error Value')
axes[3].set_title('Comparison of Max Error Values by Feature Set and Model (RandomizedSearchCV)')

# Bar plot for Median Absolute Error
sns.barplot(x='Model', y='median_absolute_error', hue='Feature Set', data=random_results_df, ax=axes[4])
axes[4].set_xlabel('Model')
axes[4].set_ylabel('Median Absolute Error Value')
axes[4].set_title('Comparison of Median Absolute Error Values by Feature Set and Model (RandomizedSearchCV)')

# Move the legend to the upper right corner
for ax in axes:
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

# Show the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the layout for better visibility
plt.show()


# Additional Requests (Bonus)